In [8]:
import pandas as pd
import numpy as np
output_path = './parameters'

In [9]:
# Fluxes
mob_data = pd.read_csv('https://raw.githubusercontent.com/BBCPandemic/BBCMobility/refs/heads/master/flux/england/E_total_flux_all.csv', header=None, skiprows=1)
flux_ids = mob_data[0].values
mob_data.drop(columns=0, inplace=True)
fluxes = mob_data.values
total_fluxes = fluxes.sum(axis=0)

# Census
census =pd.read_csv('lad_population_by_age.csv')
census.set_index('lad17cd', inplace=True)
populations = census.sum(axis=1).values
np.fill_diagonal(fluxes, populations - total_fluxes)

In [10]:
fluxes

array([[ 73461,      5,      6, ...,      0,      0,      0],
       [    19, 110538,     30, ...,      0,      0,      0],
       [     3,     49, 108999, ...,      0,      0,      0],
       ...,
       [     0,      0,      0, ..., 207750,      3,     81],
       [     0,      0,      0, ...,      0, 254846,    348],
       [     0,      0,      0, ...,     13,     50, 179240]], dtype=int64)

In [11]:
probability_matrix = np.matmul(fluxes, np.diag(1/fluxes.sum(axis=0)))

In [12]:
base_file = f'{output_path}/manypop'
with open(base_file + '.mg', 'w+') as f:
    f.write('#Migration_format_version 0.0.1\n')
    np.savetxt(f, probability_matrix, delimiter=' ', fmt='%.15f')

In [13]:
population_data = pd.DataFrame({'size': populations,
                                'contactDensity': np.full(shape=populations.shape, fill_value=1.00,
                                                          dtype=np.double),
                                'conDenAfterLD': np.full(shape=populations.shape, fill_value=0.2,
                                                         dtype=np.double),
                                'startLD': np.full(shape=populations.shape, fill_value=1.,
                                                   dtype=np.double),
                                'endLD': np.full(shape=populations.shape, fill_value=0.01,
                                                 dtype=np.double),
                                'samplingMultiplier': np.full(shape=populations.shape,fill_value=10.0,dtype=np.double),
                                'fullname':flux_ids})
# VGsim seems to require 3 columns to be comma-seperated, while the rest are space seperated, so must merge some columns by commas: 
population_data['id'] = range(0,populations.shape[0])
# Extract ID to Full country name to a separate table for relabelling countries after simulations
population_translation_data = population_data[['id','fullname']]
population_translation_data = population_translation_data.rename(columns={'id':'location'})
population_translation_data['size'] = populations
population_data['conDenAfterLD startLD endLD'] = population_data['conDenAfterLD'].astype(str) + "," + population_data[
    'startLD'].astype(str) + "," + population_data['endLD'].astype(str)
population_data = population_data[['id', 'size', 'contactDensity', 'conDenAfterLD startLD endLD', 'samplingMultiplier']]

In [14]:
with open(base_file + '.pp', 'w') as f:
    f.write('#Population_format_version 0.0.1\n')

population_data.to_csv(base_file + '.pp', header=True, index=None, mode="a", sep=' ')
population_translation_data.to_csv(f'{base_file}_country_ids.csv', index=None, sep=',', mode='w')